# Database Management

## Notebook for creating and editing database tables/entries

In [1]:
import sqlite3

db_dir = '/nfs/proj-repo/AAARG-dissertation'
db_name = 'sumresults.db'
db_path = db_dir + '/' + db_name

In [ ]:
def get_connection(path=db_path):
    conn = sqlite3.connect(path)  # creates db if doesn't exist
    c = conn.cursor()
    return conn, c

In [9]:
def create_tables(topic=True, techniques=True, instances=True, nuggets=True, meta=True, updates=True):
    conn, c = get_connection()
    if topic: # create topic table
        print("creating topic table...")
        c.execute("""CREATE TABLE IF NOT EXISTS topics (
            id integer PRIMARY KEY,
            title text,
            description text,
            start integer,
            end integer,
            query text,
            type text
        )""")
    if techniques:
        print("creating techniques table...")
        c.execute("""CREATE TABLE IF NOT EXISTS techniques (
            name text NOT NULL,
            description text
        )""")
    if instances:
        print("creating instances table...")
        c.execute("""CREATE TABLE IF NOT EXISTS instances (
            instance integer PRIMARY KEY AUTOINCREMENT,
            technique text,
            temporal boolean,
            start_exec datetime,
            end_exec datetime,
            FOREIGN KEY (technique) REFERENCES techniques (name)
        )""")
    
    if meta:
        print("creating meta table...")
        # put many-many foreign key for contained nuggets
        c.execute("""CREATE TABLE IF NOT EXISTS meta (
            topic_id integer,
            instance integer,
            summary text,
            streamids text,
            epoch_start integer,
            epoch_end integer,
            has_updates boolean,
            FOREIGN KEY (topic_id) REFERENCES topics (id),
            FOREIGN KEY (instance) REFERENCES instances (instance),
            PRIMARY KEY (instance, topic_id)
        )""")
    if updates:
        print("creating updates table...")
        c.execute("""CREATE TABLE IF NOT EXISTS updates (
            instance integer,
            topic_id integer,
            technique text,
            update_num integer,
            cur_text text,
            update_text text,
            epoch_start integer,
            epoch_end integer,
            FOREIGN KEY (instance) REFERENCES instances (instance),
            FOREIGN KEY (topic_id) REFERENCES topics (id)
            FOREIGN KEY (technique) REFERENCES techniques (name)
            PRIMARY KEY (instance, topic_id, update_num)
        )""")
    if nuggets:
        print("creating nuggets table...")
        c.execute("""CREATE TABLE IF NOT EXISTS nuggets (
            nugget_id text PRIMARY_KEY,
            topic_id integer,
            importance integer,
            nugget_text text,
            FOREIGN KEY (topic_id) REFERENCES topics (id)
        )""")
        # create linking table for many-many relationship
        print("creating linking nuggets table")
        c.execute("""CREATE TABLE IF NOT EXISTS nugget_instances (
            nugget_id text,
            topic_id integer,
            instance integer,
            technique text,
            update_num,
            is_update boolean,
            is_complete_summary boolean,
            found boolean,
            FOREIGN KEY (nugget_id) REFERENCES nuggets (nugget_id),
            FOREIGN KEY (topic_id) REFERENCES topics (id),
            FOREIGN KEY (instance) REFERENCES instances (instance),
            FOREIGN KEY (technique) REFERENCES techniques (name),
            FOREIGN KEY (update_num) REFERENCES updates (update_num)
        )""")
    print("committing changes")
    conn.commit()
    print("committed")
    conn.close()

create_tables()

creating topic table...
creating techniques table...
creating instances table...
creating meta table...
creating updates table...
creating nuggets table...
creating linking nuggets table
committing changes
committed


## Pre-Populating Database

In [ ]:
def populate_topics(topics_df):
    conn, c = get_connection()
    cols = ['id', 'title', 'description', 'start', 'end', 'query', 'type']
    entries = []
    for index, row in topics_df.iterrows():
        entry = []
        for col in cols:
            entry.append(row[col])
        entries.append(entry)
    
    c.executemany('insert into topics values (?,?,?,?,?,?,?)', rows)
    conn.commit()
    conn.close()
    print("added topics_df")

In [ ]:
# c.execute("""CREATE TABLE results (
#     topic_id integer,
#     summary text
# )""")